In [8]:
### IMPORTS
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, dash_table, State
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import numpy as np
import dash_leaflet as dl
import dash_leaflet.express as dlx
from datetime import datetime as dt

### VARIABLES
UPDATEINTERVAL = 1 # update the dashboard every 5 seconds
CARDCOLOR = "black" # the colour for the main body elements
HEADERCOLOR = 'dark' # the colour for the header

app = Dash(__name__)

colors = {
    'background': '#000100',
    'text': '#7FDBFF'
}

custom_colorscale = [(0, "yellow"),
                    (0.01, "yellow"),
                    (0.02, "green"), 
                    #(0.03, "green"), 
                    (0.05, "lightskyblue"), 
                    #(0.09, "lightskyblue"), 
                    (0.11, "dodgerblue"), 
                    #(0.20, "dodgerblue"), 
                    (0.24, "blue"), 
                    (0.4, "blue"),
                    (0.45, "darkblue"), 
                    (0.6, "darkblue"),
                    (1, "black")]

### IMPORT DATA FROM GOOGLE SHEETS DATABASE
# Import from google sheets to allow for live updates. append 'export?gid=0&format=csv' to end of google sheets share link
probedata = pd.read_csv('https://docs.google.com/spreadsheets/d/1Esr5-0_bBIOXqfuDlz3zbCE4T56i09j8Gf8kurw274I/export?gid=0&format=csv')

### DATA FOR MAP
marker_data = probedata
dicts = [dict(tooltip=marker_data.loc[m]["Name"], popup=marker_data.loc[m]["Name"], lat=marker_data.loc[m]["lat"], lon=marker_data.loc[m]["lon"], id=f'marker{m}') for m in range(0,marker_data.shape[0])] #
markers=dlx.dicts_to_geojson(dicts)
cluster = dl.GeoJSON(id="markers", data=markers, cluster=True, zoomToBoundsOnClick=True)

# --------------------------------------- FUNCTIONS -------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------------

# # Text Field as Sample Output (NOT IN USE)
# def drawText():
#     return html.Div([
#         dbc.Card(
#             dbc.CardBody([
#                 html.Div([
#                     html.H2("Text"),
#                 ], style={'textAlign': 'center'}) 
#             ])
#         ),
#     ])

# Draw a Bar Chart with Moisture Data
def drawFigure(data, x_data, y_data, probenumber):
    figure=px.bar(
            data, x_data, y_data, title=f"Probe {probenumber+1}: Moisture Over Time", color = 'Status', color_discrete_sequence=["navy", "blue", "crimson"]
            ).update_layout(
                template='plotly_dark',
                plot_bgcolor= 'rgba(0, 0, 0, 0)',
                paper_bgcolor= 'rgba(0, 0, 0, 0)',
                xaxis_title="Time (Days)",
                yaxis_title="Moisture Content",
                legend=dict(
                            orientation="v",
                            yanchor="bottom",
                            y=1.01,
                            xanchor="right",
                            x=1,
                            title = None
                        ),
                margin={"r":5,"t":0,"l":5,"b":15}
            )
    return figure

# Clean the raw data for plotting in the bar chart
def data_clean_for_graph(rawdata, probe_number):
    cleaned_data = rawdata.iloc[:,7:]
    cleaned_data = cleaned_data.transpose()
    cleaned_data['Index'] = np.arange(len(cleaned_data))
    cleaned_data['Status'] = np.nan
    
    for i in range(0,cleaned_data.shape[0]):
        if cleaned_data.iloc[i,probe_number] >= 80:
            cleaned_data.iloc[i,-1] = 'Fully Watered'
        elif cleaned_data.iloc[i,probe_number] < 80 and cleaned_data.iloc[i,probe_number] >= 50:
            cleaned_data.iloc[i,-1] = 'Sufficiently Watered'
        elif cleaned_data.iloc[i,probe_number] < 50:
            cleaned_data.iloc[i,-1] = 'Dry'
    return cleaned_data

# Clean the Raw data for the map to define color scale (NOT IN USE)
def data_cleaned_for_map(rawdata):
    cleaned_data = rawdata
    
    cleaned_data['CurrentStatus'] = np.nan
    for i in range(0,cleaned_data.shape[0]):
        if cleaned_data.iloc[i,-2] >= 80:
            cleaned_data.iloc[i,-1] = 'Fully Watered'
        elif cleaned_data.iloc[i,-2] < 80 and cleaned_data.iloc[i,-2] >= 50:
            cleaned_data.iloc[i,-1] = 'Sufficiently Watered'
        elif cleaned_data.iloc[i,-2] < 50:
            cleaned_data.iloc[i,-1] = 'Dry'
            
    cleaned_data['Index'] = np.arange(len(cleaned_data))
    return cleaned_data

# Generate a data table for output and interactions for the main dashboard
def generate_table(datatable, row_num):
    return dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in datatable.columns],
        data=datatable.to_dict('records'),
        style_cell={'textAlign': 'left', 'border':'rgb(224,224,224)','font_family': 'Helvetica','color':'rgb(204,204,204)'},
        style_table={'height':'28vh','overflow': 'Scroll'},
        style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white', 'border':'rgb(224,224,224)'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        },
    )

# Generate a data table for output and interactions for the secondary collapsable dashboard
def generate_table2(datatable, row_num):
    return dash_table.DataTable(
        id='table2',
        columns=[{"name": i, "id": i} for i in datatable.columns],
        data=datatable.to_dict('records'),
        style_cell={'textAlign': 'left', 'border':'rgb(224,224,224)','font_family': 'Helvetica','color':'rgb(204,204,204)'},
        style_table={'height':'28vh','overflow': 'scroll'},
        style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white', 'border':'rgb(224,224,224)'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        },
    )

# Generate a map of the probe locations with color scales for the Main dashboard
def generate_map(datatable, lat, lon):
    fig = px.scatter_mapbox(datatable, lat="lat", lon="lon", hover_name="Name", hover_data=["Tag", "City"],
                        color = 'Moist_9',size = 'Moist_9', color_continuous_scale = 'bluered_r',zoom=14, 
                        text = "Name")#, , name = "Name"

    fig.update_layout(mapbox_style="open-street-map", coloraxis_showscale=False)
    fig.update_mapboxes(center_lat = lat)
    fig.update_mapboxes(center_lon = lon)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(clickmode='event+select') #  mode='markers+text'
#     fig.add_annotation(xclick=0,
#                         yclick=0,
#                         xref="x",
#                         yref="y",
#                         text=f"Probe",
#                         showarrow=True,
#                         font=dict(
#                                 family="Courier New, monospace",
#                                 size=16,
#                                 color="#ffffff"
#                                 ),
#                       )
    return fig

def generate_map2(datacluster, lat, lon, zoom):
    return dl.Map([dl.TileLayer(), datacluster], #dl.LocateControl(options={'locateOptions': {'enableHighAccuracy': True}})
                  id = 'map', zoom=zoom, center=(lat, lon))

def generate_depth(df, num):
    fig = px.bar(x=[1], y=[-1])
    fig.add_layout_image(
            dict(
                source="https://raw.githubusercontent.com/allankgo/IGEN-430/main/soilprofilefordashboard.jpg",
                xref="x",
                yref="y",
                x=0.5,
                y=0.5,
                sizex=1,
                sizey=2,
                sizing="stretch",
                opacity=0.8,
                layer="below"))
    fig.update_layout(template="plotly_dark",margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_traces(marker_color='rgb(34,102,204)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.7)
    fig.update_yaxes(title=None, visible=True, showticklabels=True,ticks="inside",
                    tickfont=dict(family='Arial', color='grey', size=10),gridcolor='grey',
                    range=[-1.5, 0.5])
    fig.update_xaxes(title=None, visible=False, showticklabels=False,range=[0,2])
    return fig

old_click = None

# --------------------------------------- APP LAYOUT ------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------------

app = JupyterDash(external_stylesheets=[dbc.themes.SLATE])

server = app.server

app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            
            dbc.Row([
                #QR code maybe for DAID
#                 dbc.Col([
#                     dbc.Card([
#                         dbc.CardBody([
#                             dbc.CardImg(
#                             src="https://raw.githubusercontent.com/allankgo/IGEN-430/main/qrcodelinktodashboard.png"
#                             ,className = 'align-self-center', style={'height':'6vh', 'width':'6vh'}) 
#                         ])
#                     ], style={'height': '8vh'}, color=f"{HEADERCOLOR}")     
                    
#                 ], width=1),
                
                dbc.Col([
                        dbc.Card(
                            dbc.CardBody([
                                    html.H2("IGEN 430 Moisture Probes Dashboard",style={'fontSize':'3.5vh'}),
                            ], style={'textAlign': 'center','height':'8vh'}), color=f"{HEADERCOLOR}"
                        ),   
                ], width=10),
                
                dbc.Col([
                         dbc.Card(
                             dbc.CardBody([
                                html.Img(className="logo",
                                src="https://static.wixstatic.com/media/784966_f53b9ce060b34e2a8530dbb3a424097a~mv2_d_3440_1499_s_2.png/v1/fill/w_149,h_65,al_c,q_85,usm_0.66_1.00_0.01/784966_f53b9ce060b34e2a8530dbb3a424097a~mv2_d_3440_1499_s_2.webp"
                                ,style={'height':'5.5vh'})
                            ], style={'textAlign': 'center', 'height': '8vh'}) ,color=f"{HEADERCOLOR}"
                        ),
                ],width=2)
            ], align='center',justify="center",className="g-0"), 
    
            
    dbc.Collapse(
        html.Div([
        html.Br(),
                dbc.Row([
                    dbc.Col([
                        html.Div([
                            dbc.Card(
                                dbc.CardBody([
                                    dcc.Graph(id='my_graph', figure={}, style={'height':'77.5vh'})
                                ]), style={'height':'80vh'}
                            ),   
                        ])
                    ], width=7),
                    dbc.Col([
                        dbc.Row([
                            generate_table(probedata,1), 
                        ]),
                        html.Br(),
                        dbc.Row([
                            dbc.Card(
                dbc.CardBody([
                    dcc.Graph(id='bar-chart', figure={}, style={'height':'45vh'}
                            )#, style={'height':'52vh'}
                            ])
                            )


                        ])
                    ], width=5),
                ], align='center'), 
                html.Br(),

        html.Div(id='output_container', children=[]),
        #html.Br(),
        ]),
                id="collapse2",
                is_open=True,
            ),
# --------------------------------------Main Dashboard^----------------------------------------------------------------                        
    html.Div([
            dbc.Collapse(
            html.Div([
                
                    dbc.Row([
                        
                        # Left Side of Dashboard with Map, Heatmap, and Depth Plot
                        dbc.Col([
                            html.Div([
                                #html.Br(),
                                dbc.Card(
                                    dbc.CardBody([
                                       html.Div(children =[], id='my_graph2', style={'width': '103%', 'height': '105%', 'margin': "none", "display": "block"})
                                        # dl.Map([],id='my_graph2')#, figure={}, style={'height':'77.5vh'}
                                    ]), style={'height':'50vh'}, color=f"{CARDCOLOR}"
                                ),
                                #html.Br(),
                        dbc.Row([
                            dbc.Col([
                                dbc.Card(
                                    dbc.CardBody([
                                        dcc.Graph(id='my_graph3', figure={},style={'height':'100%'},
                                                  config={'scrollZoom': False})#
                                    ]), style={'height':'32.5vh'}, color=f"{CARDCOLOR}"),   
                                
                                dbc.Card(
                                    dbc.CardBody([
                                        dcc.Slider(
                                        id='my-slider',
                                        min=0,
                                        max=9,
                                        marks={i: f'Time {i}' for i in range(10)},
                                        value=5,
                                    ),
                                ]), style={'height':'6vh'}, color="black"),
                            ], width = 8 , style={'margin-right': '0px', 'margin-left': '0px'}),
                            
                            dbc.Col([
                                dbc.Card([
                                    dbc.CardBody([
                                        dcc.Graph(id = 'my_graph4', figure={}, style={'height':'100%'})
                                    ])
                                ], style={'height':'38.5vh'},color=f"{CARDCOLOR}")
                                
                            ], width = 4, style={'margin-right': '0px', 'margin-left': '-5px'})
                        ]),

                            ]),
                        ], width=7),
                        
                        # Data Table Column
                        dbc.Col([
                            html.Br(),
                            dbc.Row([
                                html.Div(id='datatable',children=[generate_table2(probedata,1)]), 
                            ], style={'height':'25vh'}),
                            html.Br(),
                            html.Br(),
                            
                        # Info Section
                        dbc.Card([
                            dbc.Row([
                                dbc.Col([
                                dbc.Button("Refresh Map", id='refreshmap', size='sm',n_clicks=0),
                                dbc.Button("Refresh Table", id='refreshbutton', size='sm', n_clicks=0),
                                ], width=4),
                                
                                dbc.Col([
                                    html.Label('Filter Data by Moisture Level:'),
                                ], width=4),
                                
                                dbc.Col([
                                    dcc.Checklist(
                                    options=[
                                        {'label': 'Low', 'value': 0},
                                        {'label': 'Medium', 'value': 1},
                                        {'label': 'High', 'value': 2}
                                    ],
                                value=[0,1,2],inputStyle={"margin-right": "0px", 'margin-left':'5px'}),
                                ], width=4),
                                
                            ]),
                            
                            dbc.Row([
                                html.Br(),
                                html.H4(id='output_container2', children=[]),
                                html.Br(),
                                html.Div(id='slider-output-container'),
                                html.Div(id='clickdata', children=[]),
                            
                                html.Div([
                                    html.Div(id='live-update-text'),
                                    dcc.Interval(
                                        id='interval-component',
                                        interval=UPDATEINTERVAL*1000, # in milliseconds
                                        n_intervals=0)])
                                ]),
                            ], style={'height':'18.5vh'}),
                            
                            
                            dbc.Card(
                                dbc.CardBody([
                                    dcc.Graph(id='bar-chart2', figure={}, style={'height':'90%'})
                                ]), style={'height':'43.5vh'},color=f"{CARDCOLOR}"),
                                            
                                ], width=5),
                                ], align='left'), 
            ]),
                id="collapse1",
                is_open=True,
            ),
        
        dbc.Row([
            dbc.Col([
                dbc.Button(
                "Swap Dashboard Layout",
                id="collapse-button",
                className="mb-3",
                color="primary",
                n_clicks=0,
            ),
            ], width=2)
        ], justify='end')

        ]),
 
#-------------------------------------Second Collapsable Dashboard^----------------------------------------------------
    html.Div(
    [
        dbc.Button("Open Initialization Message", id="open", n_clicks=1),
        dbc.Modal(
            [
                dbc.ModalHeader(dbc.ModalTitle("Welcome.")),
                dbc.ModalBody("Please click a row in the data table to get started! \nThis dashboard is currently in" 
                              " progress while we work on hardware and communications but please feel free to explore!"
                              " The datatable and map are fully interactive, while the slider in the bottom left controls the heatmap graphic."
                              " The two graphs in the lower right have some basic zoom functions but are otherwise incomplete."
                              " If you wish to reopen this message there is a button at the bottom of the page."),
                dbc.ModalFooter(
                    dbc.Button(
                        "Close", id="close", className="ms-auto", n_clicks=0
                    )
                ),
            ],
            id="modal",
            is_open=False,
        ),
    ]
),          

],style={'backgroundColor': colors['background']})
    )]
)
  

# ----------------------------------------- CALLBACKS -----------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------------

# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_graph', component_property='figure'),
     Output('table', 'active_cell'),
     Output('table', 'selected_cells'),
     Output('my_graph', 'clickData')],
    [Input(component_id='table', component_property='active_cell'),
     Input('my_graph', 'clickData')]
)
def update_graph(number,click):
    if click is not None and number is None:
        container = "The row chosen by user was: {}!!".format(click)
        
        latitude = click['points'][0]['lat']
        longitude = click['points'][0]['lon']
        dict = {
          "row": click['points'][0]['pointNumber'] ,
          "column": 0,
        }

        fig = generate_map(probedata, latitude, longitude)
        #fig = fig.update_mapboxes(center_lat = latitude,center_lon = longitude)
        return container, fig, dict, [], None
    elif number is not None:
        num = number['row']
        container = "The row chosen by user was: {}!!".format(num)

        latitude = probedata.loc[num,'lat']
        longitude = probedata.loc[num, 'lon']
        
        fig = generate_map(probedata, latitude, longitude)
        #fig = fig.update_mapboxes(center_lat = latitude,center_lon = longitude)
        return container, fig, number, [], None
    else:
        container = "The row chosen by user was: {}".format("Click Something")
        latitude = 49.257610
        longitude = -123.231972
        fig = generate_map(probedata, latitude, longitude)
        dict = {
          "row": 1,
          "column": 1
        }
    return container, fig, dict, [], None

@app.callback(
    Output("bar-chart", "figure"), 
    [Input("table", "active_cell")])
def update_bar_chart(number):
    if number is not None:
        y = number['row']
        x = 'Index'
        data = data_clean_for_graph(probedata, y)
        fig = drawFigure(data,x,y,y)
        return fig
    else:
        y = 0
        x = 'Index'
        data = data_clean_for_graph(probedata, y)
        fig = drawFigure(data,x,y,y)
        return fig
    
@app.callback(
    Output("table", "style_data_conditional"),
    Input("table", "active_cell"),
)
def style_selected_rows(active_cell):
    if active_cell is None:   
        row = 1
        val = [{'if': {
                'filter_query': f'{{Name}} contains "{row}"'
            },
            'backgroundColor': '#0074D9',
            'color': 'white'
        }] +[{
                    "if": {"state": "selected"},              # 'active' | 'selected'
                    "backgroundColor": "#0074D9",
                    "border": "1px solid blue",
                },]
        return val
    else:
        row = active_cell['row'] + 1
        val = [{'if': {
                'filter_query': f'{{Name}} contains "{row}"'
            },
            'backgroundColor': '#0074D9',
            'color': 'white'
        }] +[{
                    "if": {"state": "selected"},              # 'active' | 'selected'
                    "backgroundColor": "#0074D9",
                    "border": "1px solid blue",
                },]
        return val
    
# ------------------------------------------Main Callbacks^------------------------------------------------------------
@app.callback(
    [Output(component_id='output_container2', component_property='children'),
     Output(component_id='my_graph2', component_property='children'),
     Output('my_graph2', 'clickData'),
     Output('refreshmap', 'n_clicks')],
    [Input(component_id='table2', component_property='active_cell'), 
     Input('my_graph2','clickData'),
     Input('refreshmap',"n_clicks")]
)
def update_graph(number,click, n):
    if number is not None and n == 0:
        num = number['row']
        container = f"{probedata['Name'][num]} Status:"

        latitude = probedata.loc[num,'lat']
        longitude = probedata.loc[num, 'lon']
        
        fig = generate_map2(cluster, latitude, longitude, 20)
        return container, fig, None, 0
    elif click is None and n == 0:
        container = "Select a Probe from the Table or on the Map"
        latitude = 49.257610
        longitude = -123.231972
        fig = generate_map2(cluster, latitude, longitude, 13)
        dict = {
          "row": 0,
          "column": 0,
        }
        return container, fig, None, 0
    elif n != 0:
        container = "Select a Probe from the Table or on the Map"
        latitude = 49.257610
        longitude = -123.231972
        fig = generate_map2(cluster, latitude, longitude, 11)
        dict = {
          "row": 0,
          "column": 0,
        }
        return container, fig, None, 0
        

    
@app.callback(
    Output("bar-chart2", "figure"), 
    [Input("table2", "active_cell")])
def update_bar_chart(number):
    if number is not None:
        y = number['row']
        x = 'Index'
        data = data_clean_for_graph(probedata, y)
        fig = drawFigure(data,x,y,y)
        return fig
    else:
        y = 0
        x = 'Index'
        data = data_clean_for_graph(probedata, y)
        fig = drawFigure(data,x,y,y)
        return fig


# heatmap plot
@app.callback(
    [Output('slider-output-container', 'children'),
     Output('my_graph3', 'figure')],
    [Input('my-slider', 'value'),
     Input('table2', 'active_cell')])
def update_output(value, cell):
    if cell is not None:
        latitude = probedata['lat'].iloc[cell['row']]
        longitude = probedata['lon'].iloc[cell['row']]
        fig = px.density_mapbox(probedata, lat='lat', lon='lon', z=f'Moist_{value}', radius=40,
                                center=dict(lat=latitude, lon=longitude), zoom=17.3,
                                mapbox_style="open-street-map",opacity = 0.8,
                                color_continuous_scale=custom_colorscale,range_color = (0,100))
        fig.update_layout(coloraxis_showscale=False,margin={"r":0,"t":0,"l":0,"b":0})
        return 'The heatmap is currently on time "{}"'.format(value), fig
    else:
        latitude = probedata['lat'].iloc[0]
        longitude = probedata['lon'].iloc[0]
        fig = px.density_mapbox(probedata, lat='lat', lon='lon', z=f'Moist_{value}', radius=40,
                                center=dict(lat=latitude, lon=longitude), zoom=17.3,
                                mapbox_style="open-street-map",opacity = 0.8,
                                color_continuous_scale=custom_colorscale,range_color = (0,100))
        fig.update_layout(coloraxis_showscale=False,margin={"r":0,"t":0,"l":0,"b":0})
        return 'The heatmap is currently on time "{}"'.format(value), fig
        

# depth plot
@app.callback(
    Output('my_graph4', 'figure'),
    [Input('table2', 'active_cell'),
     Input('my-slider', 'value')])
def update_depthprofile(probe, value):
    if probe is not None:
        random = probe
        probenum = value
        fig = generate_depth(probedata,probenum)
        return fig
    else:
        fig = generate_depth(probedata, 1)
        return fig

@app.callback([Output("table2", "active_cell"),
               Output("table2","selected_cells"),
               Output('clickdata', 'children')],
              [Input("markers", "click_feature"),
               Input("table2", 'active_cell')])
def marker_click(marker, currentcell):
#     if currentcell is None:
#         dict = {
#             'row': 0,
#             'column': 0,
#         }
#         return dict, [], None
#     if marker is None and currentcell is None:
#         dict = {
#             'row': 0,
#             'column': 0,
#         }
#         return dict, dict, None
    if marker is not None:
        lon = marker['geometry']['coordinates'][0]
        lat = marker['geometry']['coordinates'][1]
        clickedmarker = int(marker['properties']['popup'][-2:])
        dict = {
            'row': clickedmarker-1,
            'column': 0,
        }

        #fig = generate_map2(cluster, lat, lon, 20)
        return dict, [], f"Hello from latitude {lat} and probe {marker}"

    
@app.callback(Output('live-update-text', 'children'),
              Input('interval-component', 'n_intervals'))
def update_metrics(n):
    global probedata
    probedata = pd.read_csv('https://docs.google.com/spreadsheets/d/1Esr5-0_bBIOXqfuDlz3zbCE4T56i09j8Gf8kurw274I/export?gid=0&format=csv')

    time = dt.now()
    style = {'padding': '5px', 'fontSize': '16px'}
    return [
        html.H4(f'The time is: {time}', style=style),
    ]

@app.callback(
    Output("table2", "style_data_conditional"),
    Input("table2", "active_cell"),
)
def style_selected_rows(active_cell):
    if active_cell == None:   
        row = 1
        val = [{'if': {
                'filter_query': f'{{Name}} contains "{row}"'
            },
            'backgroundColor': '#0074D9',
            'color': 'white'
        }] +[{
                    "if": {"state": "selected"},              # 'active' | 'selected'
                    "backgroundColor": "#0074D9",
                    "border": "#0074D9",
                },]
        return val
    if active_cell is not None:
        row = active_cell['row'] + 1
        val = [{'if': {
                'filter_query': f'{{Name}} contains "{row}"'
            },
            'backgroundColor': '#0074D9',
            'color': 'white'
        }] +[{
                    "if": {"state": "selected"},              # 'active' | 'selected'
                    "backgroundColor": "#0074D9",
                    "border": "#0074D9",
                },]
        return val
    
# table refresh
@app.callback(
    [Output("datatable", "children"), Output('refreshbutton','n_clicks')], 
     [Input("refreshbutton", "n_clicks")]
)
def on_button_click(n):
    if n is None:
        return generate_table2(probedata,None), None
    else:
        return generate_table2(probedata,None), None
    
@app.callback(
    Output("modal", "is_open"),
    [Input("open", "n_clicks"), Input("close", "n_clicks")],
    [State("modal", "is_open")],
)
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open
# ------------------------------- Secondary Dashboard Callbacks^-------------------------------------------------------
@app.callback(
    [Output("collapse1", "is_open"),
     Output('collapse2','is_open')],
    [Input("collapse-button", "n_clicks")],
    [State("collapse1", "is_open")],
)
def toggle_collapse(n, is_open):
    if n:
        return not is_open, is_open
    return is_open, not is_open

# ----------------------------------------- RUN THE APP ---------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------------------------

if __name__ == '__main__':
    app.run_server(debug=False)
    

127.0.0.1 - - [19/Feb/2022 16:41:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:17] "GET /_shutdown_3e1171b4-9634-452d-9104-3bd64bef9f9e HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2022 16:41:17] "GET /_alive_3e1171b4-9634-452d-9104-3bd64bef9f9e HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [19/Feb/2022 16:41:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:41:18] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - -

127.0.0.1 - - [19/Feb/2022 16:42:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2022 16:42:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2022 16:42:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:24] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2022 16:42:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:42:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [19/Feb/2022 16:43:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:43:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [19/Feb/2022 16:45:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:42] "POST /_dash-update-component HTTP/1.1" 200 -


In [9]:
import dash_html_components as html
import dash_leaflet as dl
from dash import Dash
from dash.dependencies import Output, Input
from dash_extensions.javascript import assign

# Color the feature saved in the hideout prop in a particular way (grey).
style_handle = assign("""function(feature, context){
    const match = context.props.hideout &&  context.props.hideout.properties.name === feature.properties.name;
    if(match) return {weight:5, color:'#666', dashArray:''};
}""")
# Create example app.
app = Dash()
app.layout = html.Div([
    dl.Map(center=[39, -98], zoom=4, children=[
        dl.TileLayer(),
        dl.GeoJSON(url="/assets/us-states.pbf", format="geobuf", id="states", 
                   options=dict(style=style_handle), hideout=dict(click_feature=None))
    ], style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"}, id="map"),
])
# Update the feature saved on the hideout prop on click.
app.clientside_callback("function(feature){return feature}", Output("states", "hideout"), [Input("states", "click_feature")])

if __name__ == '__main__':
    app.run_server()

C:\Users\allan\AppData\Local\Temp/ipykernel_5280/954865871.py:1: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



ModuleNotFoundError: No module named 'dash_extensions'

127.0.0.1 - - [19/Feb/2022 16:45:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:45:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:46:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:46:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:46:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:46:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:46:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [19/Feb/2022 16:49:06] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2022 16:49:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:49:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [19/Feb/2022 16:52:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2022 16:52:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.

In [11]:
# to create the requirements.txt file
import session_info
session_info.show()

127.0.0.1 - - [30/Nov/2021 11:25:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Nov/2021 11:25:29] "POST /_d

In [18]:
import dash
import dash_html_components as html
import dash_leaflet as dl
from dash.dependencies import Input, Output

positions = [(11, 11), (33, 33), (55, 55)]
markers = [dl.Marker(dl.Tooltip("test"), position=pos, id="marker{}".format(i)) for i, pos in enumerate(positions)]
cluster = dl.MarkerClusterGroup(id="markers", children=markers, options={"polygonOptions": {"color": "red"}})
print(cluster)
# app = dash.Dash(prevent_initial_callbacks=True)
# app.layout = html.Div([
#     html.Div(dl.Map([dl.TileLayer(), cluster], center=(33, 33), zoom=3, id="map",
#                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})),
#     html.Div(id='clickdata')
# ])


# @app.callback(Output("clickdata", "children"),
#               [Input(marker.id, "n_clicks") for marker in markers])
# def marker_click(*args):
#     marker_id = dash.callback_context.triggered[0]["prop_id"].split(".")[0]
#     return "Hello from {}".format(marker_id)


# if __name__ == '__main__':
#     app.run_server()


MarkerClusterGroup(children=[Marker(children=Tooltip('test'), id='marker0', position=(11, 11)), Marker(children=Tooltip('test'), id='marker1', position=(33, 33)), Marker(children=Tooltip('test'), id='marker2', position=(55, 55))], id='markers', options={'polygonOptions': {'color': 'red'}})


In [14]:
print(dt.now())

2022-01-04 11:02:00.619357


127.0.0.1 - - [04/Jan/2022 11:02:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 11:02:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.